In [ ]:
def build_momentum_query(
    table_id: str,
    feature_names: list[str],
    max_delta: int = 3,
) -> str:
    """
    Genera una query de BigQuery que calcula el momentum ponderado
    para los deltas de las columnas indicadas.

    Ejemplo de nombres de columnas:
        edad_delta_1, edad_delta_2, ..., edad_delta_max_delta

    Args:
        table_id: ID completo de la tabla en BigQuery, ej: "proyecto.dataset.c02".
        feature_names: lista de features base, ej: ["edad", "altura"].
        max_delta: cantidad de deltas disponibles (ej. 3 → _delta_1.._delta_3).

    Returns:
        str: query SQL lista para ejecutar en BigQuery.
    """

    momentum_exprs = []

    for feat in feature_names:
        terms = []
        # pesos: max_delta para delta_1, ..., 1 para delta_max_delta
        for k in range(1, max_delta + 1):
            weight = max_delta - k + 1
            col_name = f"{feat}_delta_{k}"
            terms.append(f"{weight} * {col_name}")

        expr = " + ".join(terms)
        alias = f"{feat}_momentum_ponderado"
        momentum_exprs.append(f"  {expr} AS {alias}")

    momentum_block = ",\n".join(momentum_exprs)

    query = f"""
SELECT
  *,
{momentum_block}
FROM `{table_id}`
"""
    return query.strip()
